<a href="https://colab.research.google.com/github/tanayd255/CFD/blob/main/MILP_for_PTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install gurobipy
import gurobipy as gp

params = {
    "WLSACCESSID": "d137fee5-0ca0-4c4a-9978-49f9ebf182e6",
    "WLSSECRET": "fd781dd4-faec-45c8-98fc-649860cbd274",
    "LICENSEID": 2528348,
}
env = gp.Env(params=params)
model= gp.Model(env=env)

import numpy as np
import matplotlib as plt
import scipy as sp
import pandas as pd

from gurobipy import GRB

#s is the matrix of the distances between any two nodes on the graph. First row & column (index 0) are measured from the DC
s = [[0.0,60.12,62.48,64.81,106.91,74.37,86.4,69.21],[60.12,0.0,13.91,4.76,52.14,15.62,29.3,9.34],[62.48,13.91,0.0,15.25,60.72,24.75,37.97,18.19],[64.81,4.76,15.25,0.0,47.87,11.14,24.93,4.6],[106.91,52.14,60.72,47.87,0.0,36.77,22.98,43.56],[74.37,15.62,24.75,11.14,36.77,0.0,13.8,6.82],[86.4,29.3,37.97,24.93,22.98,13.8,0.0,20.57],[69.21,9.34,18.19,4.6,43.56,6.82,20.57,0.0]]

#DMax is the carrying capacities of the 3 vehicle types in cbm
DMax = [3.0,6.0,15.0]

#c is the matrix of converting each distance to currency (e.g.- PHP) based on the vehicle type deployed between those two nodes.
c = [10.0,15.0,35.0]

#cnew is the cost of deploying any new vehicle
cnew = 2000.0

#d is the demand per node of the graph
d = [0,10,10,10,10,10,10,10]

#M is the max number of nodes allowed on one ride of one vehicle
M = 2.0

#L is the max distance which any vehicle can cover in a single ride (i.e. in one day) due to urban traffic congestion
L = 120.0

# x[k][i][j] defines whether a vehicle of the kth type traversed between nodes i & j.
#The first index/ axis (sheets before rows & columns) is the index for vehicle type.
x = model.addVars(3,8,8,vtype = GRB.BINARY,name='x')

#Need y to multiply x matrix with distances
y = model.addVars(3,8,8)

#Need z to multiply y with cost per distance as per each vehicle type
z = model.addVars(3,8,8)

#v is the number of vehicles of each type used, here there are 3 types of vehicles
v = model.addVars(3,vtype=GRB.BINARY,name='v')

#Need w to multiply the column-wise sum of x by demand to get total demand carried by a vehicle
w = model.addVars(3,8)

#Need u to sum the total number of times a node has been accessed
u = model.addVars(8)

#No ride from the same node to same node and no ride to the DC
for k in range(3):
  for i in range(8):
    model.addConstr(x[k,i,i] == 0)
    model.addConstr(x[k,i,0] == 0)


#For any edge, a vehicle of type-k can traverse iff a vehicle of the same type has reached
for k in range(3):
  for i in range(1, 8):
    for j in range (1,8):
      model.addConstr(x[k,i,j] <= x[k].sum(k,"*",i))


#For all nodes which are not 0 (i.e. the DC), the total indexes incoming are 1, so that 1 vehicle of a certain type from 1 node visits this node
#Receiving is defined with columns, i.e. the 3rd axis for any node.
model.addConstr((x.sum(axis=0)).sum(axis=0) == u)

for j in range(1,8):
  model.addConstr(u[j] == 1)

#Number of vehicles of a particular type OUT from the DC = Number of vehicles of that type IN the graph
for k in range(3):
  model.addConstr(x[k].sum(axis=1)[0] == v[k])


#Sum of columns in any sheet (i.e. vehicle type) gives which nodes it's servicing. This multiplied by the demand weight of that node, gives the total weight that vehicle is carrying.
#The total weight is summed per vehicle type and compared to the vehicle's carrying capacity
for k in range(3):
  model.addConstr(x[k].sum(axis=0) *d[j] == w[k])

for k in range(3):
    model.addConstr(w[k].sum <= v[k] * DMax[k])


#The total distance travelled by any vehicle is less than the max permissible distance
for k in range(3):
  for i in range(8):
    for j in range(8):
      model.addConstr(x[k,i,j].item() * s[i,j] == y[k,i,j])

for k in range(3):
  model.addConstr(y.sum(axis=(1,2)) <= v[k] * L)


#Constrain the number of drops executed by a vehicle type
for k in range(3):
  model.addConstr(x[k].sum <= v[k] * M)


#Define costs
for k in range(3):
  for i in range(8):
    for j in range(8):
      model.addConstr(y[k,i,j] * c[k] == z[k,i,j])


model.setObjective(z.sum + cnew * x.sum, GRB.MINIMIZE)

model.Optimize()


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528348
Academic license 2528348 - for non-commercial use only - registered to ta___@mit.edu


KeyError: 0

In [12]:
import numpy as np

xdash = np.array([[[2,2],[2,2]],[[2,2],[5,5]],[[3,3],[2,2]]])
vdash = np.array([0,0,0])

np.sum(xdash[1])

14